# Build load profile for Alameda from a series of parquet files
TODO: Switch this to use baseline data instead of Upgrade 9, as I am currently using. 

In [13]:
import os
import pandas as pd

scenario = "scenario-baseline"
housing_type = "single-family-detached"


In [15]:
directory = f"alameda/{scenario}/{housing_type}/step1_buildings/"
filepath = directory + "2706-0.parquet"

building = pd.read_parquet(filepath)

out_columns = [col for col in building.columns if col.startswith('out.electricity.')]

building.head()

,timestamp,out.electricity.ceiling_fan.energy_consumption,out.electricity.ceiling_fan.energy_consumption_intensity,out.electricity.clothes_dryer.energy_consumption,out.electricity.clothes_dryer.energy_consumption_intensity,out.electricity.clothes_washer.energy_consumption,out.electricity.clothes_washer.energy_consumption_intensity,out.electricity.cooling.energy_consumption,out.electricity.cooling.energy_consumption_intensity,out.electricity.cooling_fans_pumps.energy_consumption,...,out.total.lrmer_midcase_15__electricity.co2e_kg,out.total.lrmer_midcase_15__fuel_oil.co2e_kg,out.total.lrmer_midcase_15__natural_gas.co2e_kg,out.total.lrmer_midcase_15__propane.co2e_kg,out.total.lrmer_midcase_15.co2e_kg,out.total.lrmer_midcase_25__electricity.co2e_kg,out.total.lrmer_midcase_25__fuel_oil.co2e_kg,out.total.lrmer_midcase_25__natural_gas.co2e_kg,out.total.lrmer_midcase_25__propane.co2e_kg,out.total.lrmer_midcase_25.co2e_kg
bldg_id,,,,,,,,,,,,,,,,,,,,,
2706,2018-01-01 00:15:00,0.005,0.000016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012760,0.0,0.074611,0.0,0.087371,0.009276,0.0,0.074611,0.0,0.083887
2706,2018-01-01 00:30:00,0.005,0.000016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012760,0.0,0.000000,0.0,0.012760,0.009276,0.0,0.000000,0.0,0.009276
2706,2018-01-01 00:45:00,0.005,0.000016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012760,0.0,0.000000,0.0,0.012760,0.009276,0.0,0.000000,0.0,0.009276
2706,2018-01-01 01:00:00,0.005,0.000016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012760,0.0,0.000000,0.0,0.012760,0.009276,0.0,0.000000,0.0,0.009276
2706,2018-01-01 01:15:00,0.001,0.000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006509,0.0,0.000000,0.0,0.006509,0.004726,0.0,0.000000,0.0,0.004726


Check for missing timestamps

### List of end uses to aggregate

In [18]:
heating = [
    'out.electricity.heating.energy_consumption',
    # 'out.electricity.heating.energy_consumption_intensity',
    'out.electricity.heating_fans_pumps.energy_consumption',
    # 'out.electricity.heating_fans_pumps.energy_consumption_intensity',
    'out.electricity.heating_hp_bkup.energy_consumption',
    # 'out.electricity.heating_hp_bkup.energy_consumption_intensity',
]

cooling = [
    'out.electricity.cooling.energy_consumption',
    # 'out.electricity.cooling.energy_consumption_intensity',
    'out.electricity.cooling_fans_pumps.energy_consumption',
    # 'out.electricity.cooling_fans_pumps.energy_consumption_intensity'
]

cooking = [
    'out.electricity.range_oven.energy_consumption',
    # 'out.electricity.range_oven.energy_consumption_intensity'
]

hot_water = [
    'out.electricity.hot_water.energy_consumption',
    # 'out.electricity.hot_water.energy_consumption_intensity'
]

appliances = [
    'out.electricity.ceiling_fan.energy_consumption',
    # 'out.electricity.ceiling_fan.energy_consumption_intensity',
    'out.electricity.clothes_dryer.energy_consumption',
    # 'out.electricity.clothes_dryer.energy_consumption_intensity',
    'out.electricity.clothes_washer.energy_consumption',
    # 'out.electricity.clothes_washer.energy_consumption_intensity',
    'out.electricity.dishwasher.energy_consumption',
    # 'out.electricity.dishwasher.energy_consumption_intensity',
    'out.electricity.lighting_exterior.energy_consumption',
    # 'out.electricity.lighting_exterior.energy_consumption_intensity',
    'out.electricity.lighting_garage.energy_consumption',
    # 'out.electricity.lighting_garage.energy_consumption_intensity',
    'out.electricity.lighting_interior.energy_consumption',
    # 'out.electricity.lighting_interior.energy_consumption_intensity',
    'out.electricity.mech_vent.energy_consumption',
    # 'out.electricity.mech_vent.energy_consumption_intensity',
    'out.electricity.refrigerator.energy_consumption',
    # 'out.electricity.refrigerator.energy_consumption_intensity'
]

misc = [
    'out.electricity.heating_hp_bkup_fa.energy_consumption',
    # 'out.electricity.heating_hp_bkup_fa.energy_consumption_intensity'
    'out.electricity.freezer.energy_consumption',
    'out.electricity.freezer.energy_consumption_intensity',
    'out.electricity.permanent_spa_heat.energy_consumption',
    'out.electricity.permanent_spa_heat.energy_consumption_intensity',
    'out.electricity.permanent_spa_pump.energy_consumption',
    'out.electricity.permanent_spa_pump.energy_consumption_intensity',
    'out.electricity.plug_loads.energy_consumption',
    'out.electricity.plug_loads.energy_consumption_intensity',
    'out.electricity.pool_heater.energy_consumption',
    'out.electricity.pool_heater.energy_consumption_intensity',
    'out.electricity.pool_pump.energy_consumption',
    'out.electricity.pool_pump.energy_consumption_intensity',
    'out.electricity.well_pump.energy_consumption',
    'out.electricity.well_pump.energy_consumption_intensity'
]

total = [
    'out.electricity.net.energy_consumption',
    # 'out.electricity.net.energy_consumption_intensity',
    'out.electricity.total.energy_consumption',
    # 'out.electricity.total.energy_consumption_intensity'
]

In [17]:
# Define the end uses included in the average
end_uses = heating + cooling + cooking + hot_water + appliances

all_data = pd.DataFrame()

for file_name in os.listdir(directory):
    file_path = os.path.join(directory, file_name)
    if file_path.endswith('.parquet'):
        data = pd.read_parquet(file_path)
        
        if 'timestamp' in data.columns and all(col in data.columns for col in end_uses):
            # Ensure the timestamp column is a datetime type
            data['timestamp'] = pd.to_datetime(data['timestamp'])
            
            # Filter to only the columns of interest including timestamp
            data = data[['timestamp'] + end_uses]
            
            # Append the data to the all_data DataFrame
            all_data = pd.concat([all_data, data], axis=0, ignore_index=True)

# Group all_data by 'timestamp' and calculate the mean across all buildings
average_profile = all_data.groupby('timestamp')[end_uses].mean()

# Create a date range covering the data range
full_year = pd.date_range(
    start=all_data['timestamp'].min(), 
    end=all_data['timestamp'].max(), 
    freq='H'
)

# Reindex the average_profile DataFrame to have a row for each hour in the data range
average_profile = average_profile.reindex(full_year)

# Calculate the total load across all end uses per hour
average_profile['total_load'] = average_profile[end_uses].sum(axis=1)

# Reset the index to have 'timestamp' as a column
average_profile.reset_index(inplace=True)
average_profile.rename(columns={'index': 'timestamp'}, inplace=True)

average_profile.to_csv(f"alameda/{scenario}/{housing_type}/step2_county_average_electricity_loads.csv", index=False)

average_profile.head()

,timestamp,out.electricity.heating.energy_consumption,out.electricity.heating_fans_pumps.energy_consumption,out.electricity.heating_hp_bkup.energy_consumption,out.electricity.cooling.energy_consumption,out.electricity.cooling_fans_pumps.energy_consumption,out.electricity.range_oven.energy_consumption,out.electricity.hot_water.energy_consumption,out.electricity.ceiling_fan.energy_consumption,out.electricity.clothes_dryer.energy_consumption,out.electricity.clothes_washer.energy_consumption,out.electricity.dishwasher.energy_consumption,out.electricity.lighting_exterior.energy_consumption,out.electricity.lighting_garage.energy_consumption,out.electricity.lighting_interior.energy_consumption,out.electricity.mech_vent.energy_consumption,out.electricity.refrigerator.energy_consumption,total_load
0,2018-01-01 00:15:00,0.0,0.004793,0.0,0.0,0.0,0.000754,0.0,0.002587,0.020367,0.000830,0.003764,0.006472,0.000623,0.045803,0.000433,0.014859,0.101285
1,2018-01-01 01:15:00,0.0,0.005761,0.0,0.0,0.0,0.000328,0.0,0.001902,0.025954,0.001439,0.002682,0.005587,0.000449,0.031554,0.000630,0.014200,0.090485
2,2018-01-01 02:15:00,0.0,0.007010,0.0,0.0,0.0,0.000354,0.0,0.001305,0.020423,0.000538,0.000787,0.003830,0.000102,0.011554,0.000328,0.013249,0.059479
3,2018-01-01 03:15:00,0.0,0.007239,0.0,0.0,0.0,0.000259,0.0,0.001023,0.000862,0.000534,0.000767,0.002793,0.000072,0.007066,0.000397,0.012869,0.033882
4,2018-01-01 04:15:00,0.0,0.008328,0.0,0.0,0.0,0.000059,0.0,0.000911,0.000482,0.000000,0.000757,0.002793,0.000069,0.006089,0.000498,0.012698,0.032685
